## Setup
1. Run YAP API server
1. `pip install -r requirements.txt`
1. Run NEMO API server `uvicorn api_main:app --port 8090`
1. Have a look at the swagger OpenAPI documentation by opening http://localhost:8090/docs in your browser

In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import requests

In [50]:
requests.get('http://localhost:8090/').json()

{'message': 'Please specify command in URL path in a POST request and provide some input text in the request body.',
 'available_commands': ['run_ner_model',
  'multi_align_hybrid',
  'multi_to_single',
  'morph_yap',
  'morph_hybrid',
  'morph_hybrid_align_tokens']}

In [51]:
texts = [
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.',
    "עשרות אנשים מגיעים מתאילנד לישראל.\nתופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת."
]

## Calling the API
1. All endpoints are HTTP POST
1. Request body contains a json with Hebrew `sentences` and optional `tokenized` flag for signaling whether they are tokenized or not.
1. Request URL may include further optional parameters for choosing models/scenarios (in all but `run_ner_model` there is no need to touch these)
1. Response is in the response body as a JSON 

## Run NER Model

In [52]:
headers =   {   'accept': 'application/json', 
                'Content-Type': 'application/json' }

In [53]:
%%time 
params = {
            'model_name': 'token-multi', # / 'token-single' / 'morph' 
}
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/run_ner_model', params=params, json=payload, headers=headers).json()
list(zip(res[0]['tokenized_text'], res[0]['ncrf_preds'])), list(zip(res[1]['tokenized_text'], res[1]['ncrf_preds']))

CPU times: user 3.53 ms, sys: 100 µs, total: 3.63 ms
Wall time: 43.8 ms


([('עשרות', 'O'),
  ('אנשים', 'O'),
  ('מגיעים', 'O'),
  ('מתאילנד', 'O^S-GPE'),
  ('לישראל', 'O^S-GPE'),
  ('.', 'O')],
 [('תופעה', 'O'),
  ('זו', 'O'),
  ('התבררה', 'O'),
  ('אתמול', 'O'),
  ('בוועדת', 'O^B-ORG'),
  ('העבודה', 'I-ORG^I-ORG'),
  ('והרווחה', 'I-ORG^I-ORG^E-ORG'),
  ('של', 'O'),
  ('הכנסת', 'B-ORG^E-ORG'),
  ('.', 'O')])

### multi_to_single (get token-single predictions using a token-multi model)

In [54]:
%%time
params = {
            #'multi_model_name': 'token-multi',

}
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/multi_to_single', params=params, json=payload, headers=headers).json()
res

CPU times: user 2.43 ms, sys: 1.63 ms, total: 4.06 ms
Wall time: 31.3 ms


[{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
  'tokens': [{'text': 'עשרות', 'nemo_multi_align_token': 'O'},
   {'text': 'אנשים', 'nemo_multi_align_token': 'O'},
   {'text': 'מגיעים', 'nemo_multi_align_token': 'O'},
   {'text': 'מתאילנד', 'nemo_multi_align_token': 'S-GPE'},
   {'text': 'לישראל', 'nemo_multi_align_token': 'S-GPE'},
   {'text': '.', 'nemo_multi_align_token': 'O'}]},
 {'text': 'תופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת.',
  'tokens': [{'text': 'תופעה', 'nemo_multi_align_token': 'O'},
   {'text': 'זו', 'nemo_multi_align_token': 'O'},
   {'text': 'התבררה', 'nemo_multi_align_token': 'O'},
   {'text': 'אתמול', 'nemo_multi_align_token': 'O'},
   {'text': 'בוועדת', 'nemo_multi_align_token': 'B-ORG'},
   {'text': 'העבודה', 'nemo_multi_align_token': 'I-ORG'},
   {'text': 'והרווחה', 'nemo_multi_align_token': 'E-ORG'},
   {'text': 'של', 'nemo_multi_align_token': 'O'},
   {'text': 'הכנסת', 'nemo_multi_align_token': 'S-ORG'},
   {'text': '.', 'nemo_multi_align_token': 'O

## MD + NER

### morph_yap

In [82]:
%%time
params = {
            #'morph_model_name': 'morph',
}
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/morph_yap', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 5.85 ms, sys: 727 µs, total: 6.57 ms
Wall time: 179 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'tokens': [{'text': 'עשרות',
   'morphs': [{'form': 'עשרות',
     'nemo_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'morphs': [{'form': 'אנשים',
     'nemo_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'morphs': [{'form': 'מגיעים',
     'nemo_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'morphs': [{'form': 'מ',
     'nemo_morph': 'O',
     'lemma': 'מ',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'תאילנד',
     'nemo_morph': 'S-GPE',
     'lemma': 'תאילנד',
     'pos': 'NNP',
     'feats': 'gen=F|num=S'}]},
  {'text': 'לישראל',
   'morphs': [{'form': 'ל',
     'nemo_morph': 'O',
     'lemma': 'ל',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'ישראל',
     'nemo_morph': 'S-GPE',
     'lemma': 'ישראל',
     'pos': 'N

### multi_align_hybrid

In [83]:
%%time
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/multi_align_hybrid', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 5.02 ms, sys: 788 µs, total: 5.81 ms
Wall time: 121 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'tokens': [{'text': 'עשרות',
   'morphs': [{'form': 'עשרות',
     'nemo_multi_align_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'morphs': [{'form': 'אנשים',
     'nemo_multi_align_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'morphs': [{'form': 'מגיעים',
     'nemo_multi_align_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'morphs': [{'form': 'מ',
     'nemo_multi_align_morph': 'O',
     'lemma': 'מ',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'תאילנד',
     'nemo_multi_align_morph': 'S-GPE',
     'lemma': 'תאילנד',
     'pos': 'NNP',
     'feats': 'gen=F|num=S'}]},
  {'text': 'לישראל',
   'morphs': [{'form': 'ל',
     'nemo_multi_align_morph': 'O',
     'lemma': 'ל',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'י

### morph_hybrid
The following models got the best results in our experiments.


In [84]:
%%time
params = {
            #'multi_model_name': 'token-multi',
            #'morph_model_name': 'morph',
            # 'align_tokens': False,
}
payload = { 'sentences': texts[1],
            #'tokenized': False,
          }

res = requests.post('http://localhost:8090/morph_hybrid', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 3.23 ms, sys: 2.15 ms, total: 5.38 ms
Wall time: 167 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'tokens': [{'text': 'עשרות',
   'morphs': [{'form': 'עשרות',
     'nemo_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'morphs': [{'form': 'אנשים',
     'nemo_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'morphs': [{'form': 'מגיעים',
     'nemo_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'morphs': [{'form': 'מ',
     'nemo_morph': 'O',
     'lemma': 'מ',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'תאילנד',
     'nemo_morph': 'S-GPE',
     'lemma': 'תאילנד',
     'pos': 'NNP',
     'feats': 'gen=F|num=S'}]},
  {'text': 'לישראל',
   'morphs': [{'form': 'ל',
     'nemo_morph': 'O',
     'lemma': 'ל',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'ישראל',
     'nemo_morph': 'S-GPE',
     'lemma': 'ישראל',
     'pos': 'N

### morph_hybrid_align_tokens

In [86]:
%%time
params = {
            'verbose': 1,
            'include_yap_outputs': True,
}
payload = { 'sentences': texts[1],
            #'tokenized': False,
          }

res = requests.post('http://localhost:8090/morph_hybrid_align_tokens', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 4.18 ms, sys: 2.33 ms, total: 6.52 ms
Wall time: 166 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'tokens': [{'text': 'עשרות',
   'nemo_multi': 'O',
   'nemo_multi_align_token': 'O',
   'nemo_morph_align_token': 'O',
   'morphs': [{'form': 'עשרות',
     'nemo_morph': 'O',
     'nemo_multi_align_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'nemo_multi': 'O',
   'nemo_multi_align_token': 'O',
   'nemo_morph_align_token': 'O',
   'morphs': [{'form': 'אנשים',
     'nemo_morph': 'O',
     'nemo_multi_align_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'nemo_multi': 'O',
   'nemo_multi_align_token': 'O',
   'nemo_morph_align_token': 'O',
   'morphs': [{'form': 'מגיעים',
     'nemo_morph': 'O',
     'nemo_multi_align_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'nemo_multi': 'O^S-GPE',
   'nemo_multi_align_token': 'S-GPE',
   'nemo_morph_align_token

## Working with sequence labels
`iobes` can be used to parse the predictions (`pip install iobes`)

In [59]:
from notebook_utils import *
e = EntityRenderer()

In [87]:
payload = { 'sentences': '\n'.join(texts),}
params = {
            'verbose': 0,
            #'align_tokens': False,
}
res = requests.post('http://localhost:8090/morph_hybrid', params=params, json=payload, headers=headers).json()

In [88]:
spans = get_spans(res)

In [89]:
from IPython.core.display import display, HTML
display(HTML(e.render(spans)))


In [90]:
ents = spans_to_df(spans)
ents

,sent_id,text,label,level,scenario
0,0,מירוסלב קלוזה,PER,morph,nemo_morph
1,1,תאילנד,GPE,morph,nemo_morph
2,1,ישראל,GPE,morph,nemo_morph
3,2,וועדת ה עבודה ו ה רווחה,ORG,morph,nemo_morph
4,2,ה כנסת,ORG,morph,nemo_morph
